In [51]:
import pandas as pd
import os
from datetime import datetime, timedelta
import datetime as dt 
import numpy as np

In [52]:
os.getcwd()

'/atlas/data19/guhitj/Erdos_DL/Erdos_v2/Erdos-2024-DL-Newsworthy'

In [53]:
os.chdir('/atlas/data19/guhitj/Erdos_DL/Erdos_v2/Erdos-2024-DL-Newsworthy')

In [29]:
df_articles = pd.read_csv('finetune_roberta/news_frob_wprobs_e06_val_0_32016.csv')

In [30]:
df_articles['Publishing Time'] = pd.to_datetime(df_articles['Publishing Time'], utc = True)

In [31]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63703 entries, 0 to 63702
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Publishing Time   63703 non-null  datetime64[ns, UTC]
 1   Ticker            63703 non-null  object             
 2   Sector            63703 non-null  object             
 3   Source            63703 non-null  object             
 4   Headline          63703 non-null  object             
 5   Text              63703 non-null  object             
 6   frob_sentiment    63703 non-null  float64            
 7   frob_comp         63703 non-null  float64            
 8   frob_neg          63703 non-null  float64            
 9   frob_neu          63703 non-null  float64            
 10  frob_pos          63703 non-null  float64            
 11  openai_sentiment  63703 non-null  float64            
 12  openai_score      63702 non-null  float64            
dtypes

In [32]:
df_articles.head(5)

,Publishing Time,Ticker,Sector,Source,Headline,Text,frob_sentiment,frob_comp,frob_neg,frob_neu,frob_pos,openai_sentiment,openai_score
0,2019-03-15 10:46:42+00:00,WFC,Finance,The Motley Fool,Did Wells Fargo CEO Tim Sloan Earn His $1 Mill...,We learned this week that the scandal-plagued ...,0.0,-0.149837,0.224089,0.701659,0.074252,-1.0,-0.50
1,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,1.0,0.837837,0.027414,0.107334,0.865252,1.0,0.75
2,2019-03-15 11:33:00+00:00,MA,Finance,Forbes,A Closer Look At Mastercard's Key Value Drivers,Mastercard has consistently beat street estima...,1.0,0.992171,0.000396,0.007037,0.992567,1.0,0.80
3,2019-03-15 11:52:45+00:00,BAC,Finance,Benzinga,Jim Cramer Gives His Opinion On Bank Of Americ...,"On CNBC's ""Mad Money Lightning Round"", Jim Cra...",-1.0,-0.648877,0.695015,0.258847,0.046138,1.0,0.50
4,2019-03-15 13:29:39+00:00,GOOGL,Technology,Benzinga,Uber And Waymo Seeking Outside Funding For Aut...,Commercially viable autonomous vehicle (AV) te...,0.0,0.021284,0.027136,0.924445,0.048419,0.0,0.10


In [33]:
df_articles['Sector'].unique()

array(['Finance', 'Technology', 'Healthcare'], dtype=object)

In [34]:
stock_files = {
    "MSFT": "Microsoft_stock.csv",
    "AAPL": "Apple_stock.csv",
    "NVDA": "Nvidia_stock.csv",
    "GOOGL":"Google_stock.csv",
    "AMZN": "Amazon_stock.csv",
    "JPM":  "JP_Morgan_stock.csv",
    "V":    "Visa_stock.csv",
    "MA":   "Mastercard_stock.csv",
    "BAC":  "Bank_of_America_stock.csv",
    "WFC":  "Wells_Fargo_stock.csv",
    "LLY":  "Eli_Lily.csv",
    "UNH":  "UnitedHealth.csv",
    "JNJ":  "Johnson_and_Johnson_stock.csv",
    "MRK":  "Merck.csv",
    "ABBV": "AbbVie.csv"
}

master_stock_df = pd.DataFrame()
for ticker, file in stock_files.items():
    stock_df = pd.read_csv('Stock_data/'+file)
    stock_df = pd.read_csv('Stock_data/'+file)
    stock_df['Date'] = pd.to_datetime(stock_df['Date'], utc = True)
    stock_df.rename(columns={'Date':'Date & Time'}, inplace=True)
    stock_df['Ticker'] = ticker
    master_stock_df = pd.concat([master_stock_df, stock_df], ignore_index=True)    

In [35]:
master_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Date & Time   19185 non-null  datetime64[ns, UTC]
 1   Open          19185 non-null  float64            
 2   High          19185 non-null  float64            
 3   Low           19185 non-null  float64            
 4   Close         19185 non-null  float64            
 5   Volume        19185 non-null  int64              
 6   Dividends     19185 non-null  float64            
 7   Stock Splits  19185 non-null  float64            
 8   Ticker        19185 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(6), int64(1), object(1)
memory usage: 1.3+ MB


In [36]:
#don't run this multiple times in a row, or you keep shifting the time repeatedly
master_stock_df['Opening Time'] = master_stock_df['Date & Time'] + timedelta(hours = 9, minutes = 30)
master_stock_df['Closing Time'] = master_stock_df['Date & Time'] + timedelta(hours = 16)
master_stock_df.rename(columns = {'Date & Time':'Market Date'}, inplace = True)
master_stock_df

,Market Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Opening Time,Closing Time
0,2019-03-01 05:00:00+00:00,107.265649,107.389169,106.106430,106.923584,23501200,0.0,0.0,MSFT,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
1,2019-03-04 05:00:00+00:00,107.389176,107.607720,105.279786,106.667046,26608000,0.0,0.0,MSFT,2019-03-04 14:30:00+00:00,2019-03-04 21:00:00+00:00
2,2019-03-05 05:00:00+00:00,106.657542,106.790567,105.688363,106.134941,19538300,0.0,0.0,MSFT,2019-03-05 14:30:00+00:00,2019-03-05 21:00:00+00:00
3,2019-03-06 05:00:00+00:00,106.296481,107.047123,105.878400,106.182457,17687000,0.0,0.0,MSFT,2019-03-06 14:30:00+00:00,2019-03-06 21:00:00+00:00
4,2019-03-07 05:00:00+00:00,105.849888,105.992416,104.396116,104.890205,25339000,0.0,0.0,MSFT,2019-03-07 14:30:00+00:00,2019-03-07 21:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
19180,2024-03-22 04:00:00+00:00,175.590161,177.333886,175.144328,176.798874,4084000,0.0,0.0,ABBV,2024-03-22 13:30:00+00:00,2024-03-22 20:00:00+00:00
19181,2024-03-25 04:00:00+00:00,177.343781,177.541927,176.481835,176.878128,4366400,0.0,0.0,ABBV,2024-03-25 13:30:00+00:00,2024-03-25 20:00:00+00:00
19182,2024-03-26 04:00:00+00:00,177.571651,178.106664,176.729525,177.532028,4392400,0.0,0.0,ABBV,2024-03-26 13:30:00+00:00,2024-03-26 20:00:00+00:00
19183,2024-03-27 04:00:00+00:00,178.136391,178.909173,177.155545,178.681305,5073500,0.0,0.0,ABBV,2024-03-27 13:30:00+00:00,2024-03-27 20:00:00+00:00


In [37]:
open_market_days = master_stock_df['Opening Time'].unique()
closing_market_days = master_stock_df['Closing Time'].unique()
strict_dates = master_stock_df['Market Date'].unique()
len(open_market_days), len(closing_market_days), len(strict_dates)

(1279, 1279, 1279)

In [38]:
df_articles['Publishing Time']= df_articles['Publishing Time'].sort_values()
open_day = pd.DataFrame(open_market_days, columns= ['Next Open Time'])
close_day = pd.DataFrame(closing_market_days, columns= ['Next Closing Time'])
dates = pd.DataFrame(strict_dates, columns = ['Next Market Date'])
df_int1 = pd.merge_asof(df_articles, open_day, right_on = "Next Open Time", left_on='Publishing Time', direction = 'forward')

df_int2 = pd.merge_asof(df_articles, close_day, right_on = 'Next Closing Time', left_on = 'Publishing Time', direction = 'forward')
df_int3 = pd.merge_asof(df_articles, dates, right_on = 'Next Market Date', left_on = 'Publishing Time',direction = 'forward' )
df_int3
len(df_int1), len(df_int2), len(df_int3)

(63703, 63703, 63703)

In [39]:
common = list(df_articles)
df_a = pd.merge(df_int1, df_int2, on= common)
master_articles_df = pd.merge(df_a, df_int3, on = common)
master_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63703 entries, 0 to 63702
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Publishing Time    63703 non-null  datetime64[ns, UTC]
 1   Ticker             63703 non-null  object             
 2   Sector             63703 non-null  object             
 3   Source             63703 non-null  object             
 4   Headline           63703 non-null  object             
 5   Text               63703 non-null  object             
 6   frob_sentiment     63703 non-null  float64            
 7   frob_comp          63703 non-null  float64            
 8   frob_neg           63703 non-null  float64            
 9   frob_neu           63703 non-null  float64            
 10  frob_pos           63703 non-null  float64            
 11  openai_sentiment   63703 non-null  float64            
 12  openai_score       63702 non-null  float64    

In [40]:
master_articles_df.head(5)

,Publishing Time,Ticker,Sector,Source,Headline,Text,frob_sentiment,frob_comp,frob_neg,frob_neu,frob_pos,openai_sentiment,openai_score,Next Open Time,Next Closing Time,Next Market Date
0,2019-03-15 10:46:42+00:00,WFC,Finance,The Motley Fool,Did Wells Fargo CEO Tim Sloan Earn His $1 Mill...,We learned this week that the scandal-plagued ...,0.0,-0.149837,0.224089,0.701659,0.074252,-1.0,-0.50,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
1,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,1.0,0.837837,0.027414,0.107334,0.865252,1.0,0.75,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
2,2019-03-15 11:33:00+00:00,MA,Finance,Forbes,A Closer Look At Mastercard's Key Value Drivers,Mastercard has consistently beat street estima...,1.0,0.992171,0.000396,0.007037,0.992567,1.0,0.80,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
3,2019-03-15 11:52:45+00:00,BAC,Finance,Benzinga,Jim Cramer Gives His Opinion On Bank Of Americ...,"On CNBC's ""Mad Money Lightning Round"", Jim Cra...",-1.0,-0.648877,0.695015,0.258847,0.046138,1.0,0.50,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
4,2019-03-15 13:29:39+00:00,GOOGL,Technology,Benzinga,Uber And Waymo Seeking Outside Funding For Aut...,Commercially viable autonomous vehicle (AV) te...,0.0,0.021284,0.027136,0.924445,0.048419,0.0,0.10,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00


In [ ]:
master_articles_df.to_csv('data/master_articles_frob_wprobs.csv', index=False)

In [41]:
master_stock_df.head(5)

,Market Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Opening Time,Closing Time
0,2019-03-01 05:00:00+00:00,107.265649,107.389169,106.106430,106.923584,23501200,0.0,0.0,MSFT,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
1,2019-03-04 05:00:00+00:00,107.389176,107.607720,105.279786,106.667046,26608000,0.0,0.0,MSFT,2019-03-04 14:30:00+00:00,2019-03-04 21:00:00+00:00
2,2019-03-05 05:00:00+00:00,106.657542,106.790567,105.688363,106.134941,19538300,0.0,0.0,MSFT,2019-03-05 14:30:00+00:00,2019-03-05 21:00:00+00:00
3,2019-03-06 05:00:00+00:00,106.296481,107.047123,105.878400,106.182457,17687000,0.0,0.0,MSFT,2019-03-06 14:30:00+00:00,2019-03-06 21:00:00+00:00
4,2019-03-07 05:00:00+00:00,105.849888,105.992416,104.396116,104.890205,25339000,0.0,0.0,MSFT,2019-03-07 14:30:00+00:00,2019-03-07 21:00:00+00:00


In [42]:
master_articles_df.head(5)

,Publishing Time,Ticker,Sector,Source,Headline,Text,frob_sentiment,frob_comp,frob_neg,frob_neu,frob_pos,openai_sentiment,openai_score,Next Open Time,Next Closing Time,Next Market Date
0,2019-03-15 10:46:42+00:00,WFC,Finance,The Motley Fool,Did Wells Fargo CEO Tim Sloan Earn His $1 Mill...,We learned this week that the scandal-plagued ...,0.0,-0.149837,0.224089,0.701659,0.074252,-1.0,-0.50,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
1,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,1.0,0.837837,0.027414,0.107334,0.865252,1.0,0.75,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
2,2019-03-15 11:33:00+00:00,MA,Finance,Forbes,A Closer Look At Mastercard's Key Value Drivers,Mastercard has consistently beat street estima...,1.0,0.992171,0.000396,0.007037,0.992567,1.0,0.80,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
3,2019-03-15 11:52:45+00:00,BAC,Finance,Benzinga,Jim Cramer Gives His Opinion On Bank Of Americ...,"On CNBC's ""Mad Money Lightning Round"", Jim Cra...",-1.0,-0.648877,0.695015,0.258847,0.046138,1.0,0.50,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00
4,2019-03-15 13:29:39+00:00,GOOGL,Technology,Benzinga,Uber And Waymo Seeking Outside Funding For Aut...,Commercially viable autonomous vehicle (AV) te...,0.0,0.021284,0.027136,0.924445,0.048419,0.0,0.10,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00,2019-03-18 04:00:00+00:00


In [43]:
master_stock_df.info(), master_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Market Date   19185 non-null  datetime64[ns, UTC]
 1   Open          19185 non-null  float64            
 2   High          19185 non-null  float64            
 3   Low           19185 non-null  float64            
 4   Close         19185 non-null  float64            
 5   Volume        19185 non-null  int64              
 6   Dividends     19185 non-null  float64            
 7   Stock Splits  19185 non-null  float64            
 8   Ticker        19185 non-null  object             
 9   Opening Time  19185 non-null  datetime64[ns, UTC]
 10  Closing Time  19185 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), float64(6), int64(1), object(1)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63703 entries, 0 to 63702
Data columns (

(None, None)

In [44]:
df_all = pd.merge(master_articles_df, master_stock_df,left_on = ['Next Open Time', 'Ticker'], right_on = ['Opening Time', 'Ticker'], how = 'outer' )
df_all 

,Publishing Time,Ticker,Sector,Source,Headline,Text,frob_sentiment,frob_comp,frob_neg,frob_neu,...,Market Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Opening Time,Closing Time
0,NaT,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-03-01 05:00:00+00:00,41.887973,42.097075,41.553888,42.053814,103544800,0.0,0.0,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
1,NaT,ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-03-01 05:00:00+00:00,62.740368,63.589807,62.354977,62.999920,8567900,0.0,0.0,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
2,NaT,AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-03-01 05:00:00+00:00,82.756500,83.712997,82.550003,83.586502,99498000,0.0,0.0,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
3,NaT,BAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-03-01 05:00:00+00:00,25.918994,26.201778,25.812949,25.901320,45771500,0.0,0.0,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
4,NaT,GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-03-01 05:00:00+00:00,56.549999,57.500000,56.549999,57.425999,34086000,0.0,0.0,2019-03-01 14:30:00+00:00,2019-03-01 21:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69428,NaT,MSFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-03-28 04:00:00+00:00,420.202052,421.110417,418.365369,419.962494,21871200,0.0,0.0,2024-03-28 13:30:00+00:00,2024-03-28 20:00:00+00:00
69429,NaT,NVDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-03-28 04:00:00+00:00,900.000000,913.000000,891.929993,903.559998,43521200,0.0,0.0,2024-03-28 13:30:00+00:00,2024-03-28 20:00:00+00:00
69430,NaT,UNH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-03-28 04:00:00+00:00,495.000000,495.869995,489.299988,494.700012,3820000,0.0,0.0,2024-03-28 13:30:00+00:00,2024-03-28 20:00:00+00:00
69431,NaT,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-03-28 04:00:00+00:00,277.975547,279.283124,276.608082,278.564453,5844400,0.0,0.0,2024-03-28 13:30:00+00:00,2024-03-28 20:00:00+00:00


In [45]:
df_all[df_all['Publishing Time'].notna()]

,Publishing Time,Ticker,Sector,Source,Headline,Text,frob_sentiment,frob_comp,frob_neg,frob_neu,...,Market Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Opening Time,Closing Time
150,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,1.0,0.837837,0.027414,0.107334,...,2019-03-15 04:00:00+00:00,44.428467,45.024531,44.161680,44.733707,156171600,0.0,0.0,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00
153,2019-03-15 11:52:45+00:00,BAC,Finance,Benzinga,Jim Cramer Gives His Opinion On Bank Of Americ...,"On CNBC's ""Mad Money Lightning Round"", Jim Cra...",-1.0,-0.648877,0.695015,0.258847,...,2019-03-15 04:00:00+00:00,26.016197,26.166426,25.857130,25.892477,82346200,0.0,0.0,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00
154,2019-03-15 13:29:39+00:00,GOOGL,Technology,Benzinga,Uber And Waymo Seeking Outside Funding For Aut...,Commercially viable autonomous vehicle (AV) te...,0.0,0.021284,0.027136,0.924445,...,2019-03-15 04:00:00+00:00,59.900002,60.085999,59.350498,59.514999,51886000,0.0,0.0,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00
158,2019-03-15 11:33:00+00:00,MA,Finance,Forbes,A Closer Look At Mastercard's Key Value Drivers,Mastercard has consistently beat street estima...,1.0,0.992171,0.000396,0.007037,...,2019-03-15 04:00:00+00:00,222.284859,226.112319,222.275135,224.567734,3894200,0.0,0.0,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00
164,2019-03-15 10:46:42+00:00,WFC,Finance,The Motley Fool,Did Wells Fargo CEO Tim Sloan Earn His $1 Mill...,We learned this week that the scandal-plagued ...,0.0,-0.149837,0.224089,0.701659,...,2019-03-15 04:00:00+00:00,43.222540,43.721723,43.033193,43.601231,46117300,0.0,0.0,2019-03-15 13:30:00+00:00,2019-03-15 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69305,2024-03-15 14:00:00+00:00,NVDA,Technology,The Motley Fool,3 Brilliant Reasons to Buy Nvidia Stock Right Now,Nvidia has been an incredible outperformer ove...,1.0,0.996367,0.000726,0.002180,...,2024-03-18 04:00:00+00:00,903.880005,924.049988,870.849976,884.549988,66897600,0.0,0.0,2024-03-18 13:30:00+00:00,2024-03-18 20:00:00+00:00
69306,2024-03-15 14:31:07+00:00,NVDA,Technology,Zacks Investment Research,Is It Worth Investing in Nvidia (NVDA) Based o...,The recommendations of Wall Street analysts ar...,0.0,0.008167,0.004146,0.983541,...,2024-03-18 04:00:00+00:00,903.880005,924.049988,870.849976,884.549988,66897600,0.0,0.0,2024-03-18 13:30:00+00:00,2024-03-18 20:00:00+00:00
69307,2024-03-15 15:15:00+00:00,NVDA,Technology,WSJ,"Hey ChatGPT, Why Isn't My AI Fund Up Like Nvidia?",It's harder to hitch your wagon to Wall Street...,0.0,-0.238964,0.302505,0.633953,...,2024-03-18 04:00:00+00:00,903.880005,924.049988,870.849976,884.549988,66897600,0.0,0.0,2024-03-18 13:30:00+00:00,2024-03-18 20:00:00+00:00
69308,2024-03-15 16:19:22+00:00,NVDA,Technology,The Motley Fool,"Nvidia Stock Has 12% Upside, According This Wa...",Wedbush Securities recently raised its price t...,1.0,0.998419,0.000243,0.001095,...,2024-03-18 04:00:00+00:00,903.880005,924.049988,870.849976,884.549988,66897600,0.0,0.0,2024-03-18 13:30:00+00:00,2024-03-18 20:00:00+00:00


In [46]:
df_all['Market Date'] = pd.to_datetime(df_all['Market Date'])
df_all['Market Date'] = df_all['Market Date'].dt.date
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69433 entries, 0 to 69432
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Publishing Time    63703 non-null  datetime64[ns, UTC]
 1   Ticker             69433 non-null  object             
 2   Sector             63703 non-null  object             
 3   Source             63703 non-null  object             
 4   Headline           63703 non-null  object             
 5   Text               63703 non-null  object             
 6   frob_sentiment     63703 non-null  float64            
 7   frob_comp          63703 non-null  float64            
 8   frob_neg           63703 non-null  float64            
 9   frob_neu           63703 non-null  float64            
 10  frob_pos           63703 non-null  float64            
 11  openai_sentiment   63703 non-null  float64            
 12  openai_score       63702 non-null  float64    

In [47]:
tech_tickers = ["MSFT", "AAPL", "NVDA", "GOOGL", "AMZN"]
healthcare_tickers = ["LLY", "UNH", "JNJ", "MRK", "ABBV"]
finance_tickers = ["JPM", "V", "MA", "BAC", "WFC"]

def map_sector(ticker):
    if ticker in tech_tickers:
        return "Technology"
    elif ticker in healthcare_tickers:
        return "Healthcare"
    elif ticker in finance_tickers:
        return "Finance"
    else:
        return "Unknown"

In [48]:
reorder = [
    'Publishing Time',
    'Market Date',
    'Ticker',
    'Sector',
    'frob_sentiment',
    'frob_comp',
    'frob_neg',
    'frob_neu',
    'frob_pos',
    'Source',
    'Headline',
    'Text',
    'Open',
    'High',
    'Low',
    'Close',
    'Volume',
    'Dividends',
    'Stock Splits'
]

df_all = df_all[reorder]
df_all['Sector'] = df_all['Ticker'].apply(map_sector)
df_all

/tmp/ipykernel_2129577/195815592.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Sector'] = df_all['Ticker'].apply(map_sector)


,Publishing Time,Market Date,Ticker,Sector,frob_sentiment,frob_comp,frob_neg,frob_neu,frob_pos,Source,Headline,Text,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,NaT,2019-03-01,AAPL,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.887973,42.097075,41.553888,42.053814,103544800,0.0,0.0
1,NaT,2019-03-01,ABBV,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.740368,63.589807,62.354977,62.999920,8567900,0.0,0.0
2,NaT,2019-03-01,AMZN,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.756500,83.712997,82.550003,83.586502,99498000,0.0,0.0
3,NaT,2019-03-01,BAC,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.918994,26.201778,25.812949,25.901320,45771500,0.0,0.0
4,NaT,2019-03-01,GOOGL,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.549999,57.500000,56.549999,57.425999,34086000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69428,NaT,2024-03-28,MSFT,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.202052,421.110417,418.365369,419.962494,21871200,0.0,0.0
69429,NaT,2024-03-28,NVDA,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,900.000000,913.000000,891.929993,903.559998,43521200,0.0,0.0
69430,NaT,2024-03-28,UNH,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,495.000000,495.869995,489.299988,494.700012,3820000,0.0,0.0
69431,NaT,2024-03-28,V,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277.975547,279.283124,276.608082,278.564453,5844400,0.0,0.0


In [49]:
print(df_all['frob_comp'])

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
69428   NaN
69429   NaN
69430   NaN
69431   NaN
69432   NaN
Name: frob_comp, Length: 69433, dtype: float64


In [50]:
df_all.to_csv('data/complete_next_open_frob_best.csv', index = False)